# Asthma-Like Illness Emergency Department Presentations (annual) | Processing

The main tasks completed to clean and preprocess this dataset were:

**Data Manipulation**
1. Rename columns.
2. Reformat 'financial year' values from XX/YY to XXXX/YYYY
3. Remove 'LHD' from LHD name values.
4. Remove 'All' data (Representing a state-wide average).
5. Remove columns holding Confidence Interval data.
6. Remove rows holding 'Persons' data in the sex column (Representing a genderless rate per 100,000).

## Set Up

Ensure that the required libraries are available by running the below code in the terminal before execution:
- pip install pandas


Execute the following in the jupyter notebook before execution to ensure that the required libraries are imported:

In [1]:
import pandas as pd

## Load Dataset

In [2]:
# File path.
file_path = 'data-raw.csv'

# Read the file.
df = pd.read_csv(file_path)

## Data Manipulation

Rename columns to match Air Quality data set.

In [3]:
# Rename columns.
df = df.rename(columns={
    'LHD': 'lhd',
    'Period': 'financial year'
})

# Set column names to lower case.
df.columns = df.columns.str.lower()

Mark Year data in 'financial year' column more verbose.

In [4]:
# Reformat 'financial year' values from XX/YY to XXXX/YYYY.
df['financial year'] = df['financial year'].apply(
    lambda x: f'20{x[:2]}/20{x[3:]}' if isinstance(x, str) else x
)

Remove ' LHD' for Local Health District values.

In [5]:
# Remove ' LHD' from the 'lhd' column.
df['lhd'] = df['lhd'].str.replace(' LHD', '')

Remove rows representing state-wide aggregated data.

In [6]:
# Remove rows with NaN in the 'lhd' column.
df = df.dropna(subset=['lhd'])

# Remove rows with 'All' in the 'lhd' column.
df = df[~df['lhd'].str.contains('All')]

Remove columns holding Confidence Interval data.

In [7]:
# Drop columns with '% ci' in the header
df = df.loc[:, ~df.columns.str.contains('% ci')]

Remove rows holding 'Persons' data in the sex column.

In [8]:
# Drop rows with 'Persons' in the 'sex' column.
df = df[~df['sex'].str.contains('Persons')]

## Output Processed Dataset

In [9]:
# File path.
file_path_output = 'data-processed.csv'

# Save the file.
df.to_csv(file_path_output, index=False)

## View Dataset

In [10]:
df

,sex,lhd,financial year,"rate per 100,000 population"
0,Males,Sydney,2014/2015,283.1
1,Males,Sydney,2015/2016,287.6
2,Males,Sydney,2016/2017,263.7
3,Males,Sydney,2017/2018,229.5
4,Males,Sydney,2018/2019,244.4
...,...,...,...,...
274,Females,Far West,2018/2019,884.7
275,Females,Far West,2019/2020,692.3
276,Females,Far West,2020/2021,592.1
277,Females,Far West,2021/2022,689.7
